## Character-level recurrent sequence-to-sequence model

In [27]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

### Configuration

In [28]:
batch_size = 64 #Batch size for trainning
epochs = 100 #Number of epohs to train for 
latent_dim = 256 #Latent dimensionality of the encoding space.
num_samples = 10000 #number of sampples to train
data_path = "fra-eng/fra.txt"

### Prepare the data

In [48]:
#vectorize the data
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")

for line in lines[0: 10]:
    input_text, target_text, _ = line.split('\t')
    print("input text -> ", input_text)
    print("target text -> ", target_text)
    print("underscore -> ", _)
    print("line ->",line)


input text ->  Go.
target text ->  Va !
underscore ->  CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
line -> Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
input text ->  Go.
target text ->  Marche.
underscore ->  CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
line -> Go.	Marche.	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8090732 (Micsmithel)
input text ->  Go.
target text ->  En route !
underscore ->  CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
line -> Go.	En route !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #8267435 (felix63)
input text ->  Go.
target text ->  Bouge !
underscore ->  CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
line -> Go.	Bouge !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #9022935 (Micsmithel)
input text ->  Hi.
target text ->  Salut !

In [50]:
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" charachter
    # for the targets, and "\n" as "end sequence" charachter.
    
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
            

In [53]:
print("input texts -> ", input_texts[0: 10])
print("target texts -> ", target_texts[0:10])
print("input characters -> ", input_characters)
print("target characters -> ", target_characters)

input texts ->  ['Go.', 'Go.', 'Go.', 'Go.', 'Hi.', 'Hi.', 'Run!', 'Run!', 'Run!', 'Run!']
target texts ->  ['\tVa !\n', '\tMarche.\n', '\tEn route !\n', '\tBouge !\n', '\tSalut !\n', '\tSalut.\n', '\tCours\u202f!\n', '\tCourez\u202f!\n', '\tPrenez vos jambes à vos cous !\n', '\tFile !\n']
input characters ->  {'M', 'u', 'l', 'J', '8', '?', 'x', 'g', 'v', 'k', 'A', 'H', 'T', 'n', 'P', 'j', 'R', 'C', 'b', '"', 'I', 'c', '1', 'W', ' ', '2', 'f', 'q', ',', 'a', '&', '-', 'w', 'F', '9', 'L', 'o', 'y', '5', '!', ':', 'E', 'z', 'U', 'S', '3', 'B', 'Y', 'G', 'K', 'Q', 's', 'p', '0', '.', 'i', '%', 'm', "'", 'V', 'D', 'r', 'd', 'O', 'é', 'e', 'h', '7', '$', 'N', 't'}
target characters ->  {'«', 'î', 'œ', 'A', 'H', 'â', 'R', 'I', 'W', 'a', '-', 'o', 'U', 'S', 'Ç', 'K', '\t', 'p', '\n', 'r', 'd', 'é', 'v', 'J', 'k', 'û', 'T', 'P', 'c', '1', 'q', 'F', 'y', '5', ':', '(', '3', 'B', 'Y', 'À', '.', 'i', '%', 'ç', 'e', 'f', 'h', 'N', '\u2009', 'l', '?', 'g', 'n', 'j', 'à', 'ê', ',', '&', 'L', '\u202f

In [54]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts]) # returns the num of chars from the item which has the max num of chars
max_decoder_seq_length = max([len(txt) for txt in target_texts]) # returns the num of chars from the item which has the max num of chars


In [55]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens) #len(input_characters)
print("Number of unique output tokens:", num_decoder_tokens) #len(target_characters)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)   


Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 93
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [33]:
input_token_index = dict([char, i] for i, char in enumerate(input_characters))
target_token_index = dict([char, i] for i, char in enumerate(target_characters))

In [34]:
encoder_input_data = np.zeros((len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype="float32")
decoder_input_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")
decoder_target_data = np.zeros((len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32")

In [35]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.0
    encoder_input_data[i, t + 1:, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.0
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
            if t > 0:
                # decoder_target_data will be ahead by one timestep
                # and will not include the start character.
                decoder_target_data[i, t - 1, target_token_index[char]] = 1.0
    decoder_input_data[i, t + 1:, target_token_index[" "]] = 1.0
    decoder_target_data[i, t:, target_token_index[" "]] = 1.0

### Build the model

In [36]:
# Define an input sequence and process it.
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))

# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)

### Train the model

In [37]:
# model.compile(optimizer="rmsprop", loss="categorical_crossentropy", metrics=['accuracy'])

# model.fit([encoder_input_data, decoder_input_data], decoder_target_data, batch_size=batch_size, epochs=epochs, validation_split=0.2)

# model.save("s2s")

### Run inference (sampling)
1. encode input and retrieve initial decoder state
2. run one step of decoder with this initial state and a "start of sequence" token as target. Output will be the next target token
3. Repeat with the current target token and current states

In [38]:
# Define sampling models
# Restore the model and construct the encoder and decoder.
model = keras.models.load_model('s2s')

In [39]:
encoder_inputs = model.input[0]   # input_1
encoder_outputs, state_h_enc, state_c_enc = model.layers[2].output   # lstm_1
encoder_states = [state_h_enc, state_c_enc]
encoder_model = keras.Model(encoder_inputs, encoder_states)

decoder_inputs = model.input[1]   # input_2
decoder_state_input_h = keras.Input(shape=(latent_dim,))
decoder_state_input_c = keras.Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_lstm = model.layers[3]
decoder_outputs, state_h_dec, state_c_dec = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h_dec, state_c_dec]
decoder_dense = model.layers[4]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

In [40]:
# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict((i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict((i, char) for char, i in target_token_index.items())

In [41]:
def decode_sequence(input_seq):
    #Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index["\t"]] = 1.0

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ""
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if sampled_char == "\n" or len(decoded_sentence) > max_decoder_seq_length:
            stop_condition = True

        #Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.0

        #Update states  
        states_value = [h, c]
    return decoded_sentence


    

In [44]:
# Now we can generate decoded sentences as such

for seq_index in range(3):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index : seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print("-")
    print("Input sentence:", input_texts[seq_index])
    print("Decoded sentence:", decoded_sentence)

1/1 [==============================] - 0s 35ms/step
-
Input sentence: Go.
Decoded sentence: C'est passire !

1/1 [==============================] - 0s 29ms/step
-
Input sentence: Go.
Decoded sentence: C'est passire !

1/1 [==============================] - 0s 26ms/step
-
Input sentence: Go.
Decoded sentence: C'est passire !

